In [1]:
# load model and make automarker
# TODO: mark a range of images, not all because at the begining the model have low accuracy

In [1]:
import os
import cv2
import json
import numpy as np
import tensorflow as tf
from utils import load_image
import matplotlib.pyplot as plt
from imutils.paths import list_images

In [2]:
eye_detector = tf.keras.models.load_model('eye_detector_model.hdf5')

2022-08-02 11:26:24.107089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 11:26:24.717116: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 11:26:24.719120: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 11:26:24.721204: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [3]:
f'{eye_detector.count_params():,} total params'

'80,640 total params'

In [4]:
datapath = {
    'images_path': "/home/choppy/TOC/thesis-msc-2022/data/images/aws/bonafide/unmarked",
    'via_file': None
}

images_path = sorted([*list_images(datapath.get('images_path'))])

In [5]:
# add gt as a green box

In [1]:
image = load_image(np.random.choice(images_path))
# redim image
original_shape = image.shape[:] # h, w, c = image.shape
image_input = cv2.resize(image, (224, 224))
# create new axis
image_input = image_input[np.newaxis, ...]

# format image and do inference
image_input = image_input  / 255.

pred = eye_detector.predict(image_input)[0]

h, w, c = original_shape
pred = pred * [w, h, w, h] # image width, height
pred = pred.astype(np.int32)
# pred
pt1 = pred[:2]
pt2 = pred[:2] + pred[2:]
rect_image = image = cv2.rectangle(image, pt1, pt2, [200, 50, 50], thickness=2)

fig, ax = plt.subplots(1, 1, figsize=(5, 7))

# ax.add_image(rect_image)
ax.imshow(rect_image)
ax.axis('off')

plt.show()

NameError: name 'load_image' is not defined

In [ ]:
# generate gif with random data to show how improves over time

In [8]:
# select range of images
_from = 0
_to = 50

# make via json format
via_json = {}

for i in range(_from, _to):
    impath = images_path[i]
    filename = os.path.basename(impath)
    image = load_image(impath)
    
    original_shape = image.shape[:] # h, w, c = image.shape
    image_input = cv2.resize(image, (224, 224))
    image_input = image_input[np.newaxis, ...]
    image_input = image_input  / 255.
    
    pred = eye_detector.predict(image_input)[0]
    
    h, w, c = original_shape
    pred = pred * [w, h, w, h] # image width, height
    pred = pred.astype(np.int32)
    
    x_start, y_start, x_end, y_end = pred
    x_end = x_end - x_start
    y_end = y_end - y_start
    
    # structure
    # {
    #     "M2E0M2VkO0EyMTIzMDg1.png143502": {
    #         "filename": "M2E0M2VkO0EyMTIzMDg1.png",
    #         "size": 143502,
    #         "regions": [
    #             {
    #                 "shape_attributes": {
    #                     "name": "rect",
    #                     "x": 62,
    #                     "y": 137,
    #                     "width": 106,
    #                     "height": 47
    #                 },
    #                 "region_attributes": {
    #                     "Face": "Periocular"
    #                 }
    #             }
    #         ],
    #         "file_attributes": {}
    #     },
    # }
    
    filesize = os.path.getsize(impath)
    obj = {
        "filename": filename,
        "size": filesize,
        "regions": [
            {
                "shape_attributes": {
                    "name": "rect",
                    "x": int(x_start),
                    "y": int(y_start),
                    "width": int(x_end),
                    "height": int(y_end),
                },
                "region_attributes": {
                    "Face": "Periocular"
                }
            }
        ],
        "file_attributes": {}
    }
    
    via_json[f'{filename}{filesize}'] = obj
    
    
    
#     pt1 = pred[:2]
#     pt2 = pred[2:]
#     rect_image = image = cv2.rectangle(image, pt1, pt2, [200, 50, 50], thickness=2)
    
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()

In [9]:
json.dump(via_json, open('regs.json', 'w'))